In [ ]:
!pip install paho-mqtt

In [ ]:
import paho.mqtt.client as mqtt
import time
import json
import csv

# device attribute for processing
CLIENT_ID = ""
TOKEN = ""
SECRET = ""
HOST = "mqtt.netpie.io"
PORT = 1883


TOPIC_DATA = "@msg/sensor/distance"
TOPIC_ALARM = "@msg/action/result"

standard_distance = None
TOLERANCE = 10.0
recorded_data = []

def process_data(distance, button_state):
    global standard_distance

    current_std = standard_distance if standard_distance is not None else 0
    recorded_data.append([current_std, distance])

    if button_state == 0:
        standard_distance = distance
        print(f">> 🔵 CALIBRATION MODE: Set Standard Distance = {standard_distance} cm")
        return "OFF"


    else:
        if standard_distance is None:
            print(">> ⚠️ Waiting for Calibration (Please press button)")
            return "OFF"

        diff = abs(distance - standard_distance)
        print(f"   Mode: Monitor | Current: {distance} | Ref: {standard_distance} | Diff: {diff:.2f}")

        if diff > TOLERANCE:
            return "ON"
        else:
            return "OFF"

def on_connect(client, userdata, flags, rc):
    print("Connected to NetPIE with result code " + str(rc))
    client.subscribe(TOPIC_DATA)

def on_message(client, userdata, msg):
    try:
        payload = msg.payload.decode('utf-8')
        data_json = json.loads(payload)

        current_distance = float(data_json['d'])
        current_button = int(data_json['b'])

        command = process_data(current_distance, current_button)


        client.publish(TOPIC_ALARM, command)

    except Exception as e:
        print("Error processing message:", e)

# --- 4. Main Execution ---
client = mqtt.Client(client_id=CLIENT_ID, protocol=mqtt.MQTTv311)
client.username_pw_set(TOKEN, SECRET)
client.on_connect = on_connect
client.on_message = on_message

print("Connecting to NetPIE...")
print("running... (Press Stop button to save CSV)")

try:
    client.connect(HOST, PORT, 60)
    client.loop_forever()

except KeyboardInterrupt:

    print("\n\n🛑 Stopped by User. Saving data to CSV...")

    filename = 'sensor_log.csv'

    try:
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)

            writer.writerow(['standard distance', 'real distance'])

            writer.writerows(recorded_data)

        print(f"✅ Data saved successfully to '{filename}'")
        print(f"📊 Total records: {len(recorded_data)}")

    except Exception as e:
        print(f"❌ Error saving file: {e}")

    client.disconnect()